In [ ]:
import websocket
from websocket import WebSocketApp

import json
import numpy as np
import pandas as pd
import time


inventory = {"TS": 250, "AD": 250, "TT": 250}
risk_mode = 'high'  # 'high' or 'safe'
cash = 10000
pnl_so_far = 0
last_enforcer_time = time.time()
volume_window = {"TS": [], "AD": [], "TT": []}
volume_threshold = 20
cooldown_timer = {"TS": 0, "AD": 0, "TT": 0}
cooldown_seconds = 10

recommendations = []
executed_trades = {}

def detect_price_enforcer(order_levels, threshold=35):
    for level in order_levels:
        if level.get('qty', 0) >= threshold or level.get('total_volume', 0) >= threshold:
            return level['price']
    return None

def calculate_fair_price(prices, inventory, order_book_bids, order_book_asks, asset):
    def calculate_book_anchored_price(bids, asks):
        if not bids or not asks:
            return None
        best_bid = bids[0]['price']
        best_ask = asks[0]['price']
        bid_qty = bids[0]['qty']
        ask_qty = asks[0]['qty']
        return (best_bid * bid_qty + best_ask * ask_qty) / (bid_qty + ask_qty)
    mu_ema = prices.ewm(span=8, adjust=False).mean().iloc[-1]
    mu_book = calculate_book_anchored_price(order_book_bids, order_book_asks)
    sigma = np.std(prices.diff().dropna())

    pe_bid = detect_price_enforcer(order_book_bids)
    pe_ask = detect_price_enforcer(order_book_asks)

    if pe_bid and pe_ask:
        mu_enf = (pe_bid + pe_ask) / 2
        dist = abs(mu_ema - mu_enf)
        w_enf = max(0.2, min(0.8, dist / 2))
        blended_mu = (1 - w_enf) * mu_ema + w_enf * mu_enf
    else:
        blended_mu = mu_ema

    if mu_book:
        fair_price = 0.6 * blended_mu + 0.4 * mu_book
    else:
        fair_price = blended_mu
        fair_price = mu_ema

    return fair_price, sigma

def calculate_liquidity(order_book_bids, order_book_asks):
    imbalance = sum([level['qty'] for level in order_book_bids]) - sum([level['qty'] for level in order_book_asks])
    return (imbalance, (sum([level['qty'] for level in order_book_bids]) + sum([level['qty'] for level in order_book_asks])) / 2)

def adjust_quantity(base_qty, inventory, symbol, liquidity_ratio, max_qty=40, min_qty=10):
    global risk_mode
    if risk_mode == 'high':
        base_qty *= 1.3

    available_qty = inventory[symbol]
    base_qty = max(min_qty, min(int(available_qty * 0.2), max_qty))

    position_skew = available_qty / 100
    if position_skew > 1:
        return max(min_qty, int(base_qty * 0.5))
    elif position_skew < -1:
        return min(max_qty, int(base_qty * 1.5))

    if liquidity_ratio > 200:
        return min(max_qty, base_qty + 5)
    else:
        return base_qty

def adjust_for_liquidity(strategy_decision, liquidity_ratio, prices):
    if liquidity_ratio > 200:
        spread_adjustment = 0.1
        qty_adjustment = 30
    else:
        spread_adjustment = 0.3
        qty_adjustment = 10

    new_decisions = []
    for side, price, qty in strategy_decision:
        adjusted_price = price + spread_adjustment if side == "SELL" else price - spread_adjustment
        adjusted_qty = qty + qty_adjustment
        new_decisions.append((side, adjusted_price, adjusted_qty))

    return new_decisions

def generate_layered_quotes(fair_price, base_spread, qty, layers=3, spread_step=0.1, qty_decay=0.6, include_extremes=True):
    quotes = []
    for i in range(layers):
        spread = base_spread + i * spread_step
        size = max(1, int(qty * (qty_decay ** i)))
        quotes.append(("BUY", fair_price - spread, size))
        quotes.append(("SELL", fair_price + spread, size))
    if include_extremes:
        quotes.append(("BUY", fair_price - 15, 10))  # Opportunistic low bid
        quotes.append(("SELL", fair_price + 15, 10))  # Opportunistic high ask
    return quotes

def detect_trend(prices, trend_history=None, symbol=None):
    if len(prices) < 2 or np.var(prices) == 0:
        return 0.0
    ema = prices.ewm(span=8, adjust=False).mean()
    momentum = ema.diff().mean()
    slope = np.polyfit(np.arange(len(ema)), ema, 1)[0]

    combined_signal = slope + 0.5 * momentum

    if trend_history is not None and symbol is not None:
        trend_history[symbol].append(combined_signal)
        if len(trend_history[symbol]) > 3:
            trend_history[symbol] = trend_history[symbol][-3:]
        if all(abs(s) > 0.05 for s in trend_history[symbol]) and (combined_signal * trend_history[symbol][0] > 0):
            return combined_signal
        else:
            return 0.0
    else:
        return combined_signal



def score_quote(symbol, side, price, qty, fair_price, trend, inventory_skew, avg_volume):
    global risk_mode
    score = 0
    if (trend > 0.05 and side == "SELL") or (trend < -0.05 and side == "BUY"):
        score += 2.5 if risk_mode == 'high' else 2
    if (inventory_skew > 0 and side == "SELL") or (inventory_skew < 0 and side == "BUY"):
        score += 1.5
    if avg_volume > 15:
        score += 1.5 if risk_mode == 'high' else 1
    if abs(price - fair_price) / fair_price <= 0.003:
        score += 1
    return score

trend_history = {"TS": [], "AD": [], "TT": []}

def enhanced_strategy(prices, inventory, price_enforcer_bid, price_enforcer_ask, order_book_bids, order_book_asks, asset, avg_volume, flatten_mode=False):
    fair_price, sigma = calculate_fair_price(prices, inventory, order_book_bids, order_book_asks, asset)
    imbalance, liquidity_ratio = calculate_liquidity(order_book_bids, order_book_asks)
    base_qty = 20
    qty = adjust_quantity(base_qty, inventory, asset, liquidity_ratio)
    trend_slope = detect_trend(prices, trend_history, asset)
    inventory_skew = inventory[asset] / 100

    strategy_type = "Market Making"
    if trend_slope > 0.15:
        strategy_type = "Trend Following (Strong Uptrend)"
        quotes = [("SELL", fair_price * 1.003, qty)]
    elif trend_slope > 0.05:
        strategy_type = "Trend Following (Uptrend)"
        quotes = [("SELL", fair_price * 1.002, qty)]
    elif trend_slope < -0.15:
        strategy_type = "Trend Following (Strong Downtrend)"
        quotes = [("BUY", fair_price * 0.997, qty)]
    elif trend_slope < -0.05:
        strategy_type = "Trend Following (Downtrend)"
        quotes = [("BUY", fair_price * 0.998, qty)]
    else:
        quotes = generate_layered_quotes(fair_price, sigma * 0.5, qty)
        if trend_slope > 0.1:
            quotes += [
                ("SELL", fair_price + 0.6, int(qty * 0.6)),
                ("SELL", fair_price + 1.0, int(qty * 0.4))
            ]

    if flatten_mode:
        if inventory[asset] > 0:
            quotes = [("SELL", fair_price - 0.05, inventory[asset])]
            strategy_type += " → Flattening"
        elif inventory[asset] < 0:
            quotes = [("BUY", fair_price + 0.05, abs(inventory[asset]))]
            strategy_type += " → Flattening"

    adjusted_quotes = adjust_for_liquidity(quotes, liquidity_ratio, prices)

    if imbalance > 40:
        print(f" Order Book Imbalance Detected: Favoring SELL side ({imbalance})")
    elif imbalance < -40:
        print(f" Order Book Imbalance Detected: Favoring BUY side ({imbalance})")

    enriched_quotes = []
    for side, price, qty in adjusted_quotes:
        score = score_quote(asset, side, price, qty, fair_price, trend_slope, inventory_skew, avg_volume)
        enriched_quotes.append((score, asset, side, price, qty, strategy_type))

    return adjusted_quotes, fair_price, sigma, strategy_type, trend_slope, enriched_quotes

def output_quotes(strategy_decision, symbol, fair_price, sigma, strategy_type, trend_slope, volume):
    print(f"\n{symbol} | Strategy: {strategy_type} | Fair Price: {round(fair_price, 2)} | Vol: {volume} | σ: {round(sigma, 4)} | Trend Slope: {round(trend_slope, 5)}")
    for side, price, qty in strategy_decision:
        print(f"  {side} {qty} @ {round(price, 2)}")

def on_message(ws, message):
    global last_enforcer_time, recommendations
    data = json.loads(message)
    current_time = time.time()
    recommendations = []

    for symbol in ["TS", "AD", "TT"]:
        if current_time - cooldown_timer[symbol] < cooldown_seconds:
            continue

        prices = pd.Series([item[1] for item in data["GameState"][symbol]["price_history"]])
        if prices.empty:
            print(f"No price data for {symbol}, skipping.")
            continue

        price_enforcer_bid = data["GameState"][symbol]["price_enforcer_bid"]
        price_enforcer_ask = data["GameState"][symbol]["price_enforcer_ask"]
        order_book_bids = data["GameState"][symbol]["buy_side_limit_levels"]
        order_book_asks = data["GameState"][symbol]["sell_side_limit_levels"]

        recent_volume = np.random.choice([np.random.randint(0, 20), np.random.randint(50, 100)], p=[0.8, 0.2])
        volume_window[symbol].append(recent_volume)
        if len(volume_window[symbol]) > 5:
            volume_window[symbol] = volume_window[symbol][-5:]
        avg_volume = sum(volume_window[symbol])

        enforcer_active = (current_time - last_enforcer_time < 5)
        if current_time - last_enforcer_time > 45:
            last_enforcer_time = current_time
            enforcer_active = True

        if avg_volume < volume_threshold and not enforcer_active:
            print(f"{symbol}: Skipping quoting due to low volume ({avg_volume})")
            continue

        # Detects enforcer walls dynamically
        strategy_decision, fair_price, sigma, strategy_type, trend_slope, enriched = enhanced_strategy(
            prices, inventory, price_enforcer_bid, price_enforcer_ask, order_book_bids, order_book_asks, symbol, avg_volume
        )
        enriched.sort(reverse=True, key=lambda x: x[0])
        recommendations.extend(enriched)
        output_quotes(strategy_decision, symbol, fair_price, sigma, strategy_type, trend_slope, avg_volume)
        # Simulate partial fills with variable fill rates 
        for side, price, qty in strategy_decision:
            fill_rate = np.random.uniform(0.1, 0.3)
            filled_qty = int(qty * fill_rate)
            if side == "BUY":
                inventory[symbol] += filled_qty
            elif side == "SELL":
                inventory[symbol] -= filled_qty
            elif side == "SELL":
                inventory[symbol] -= filled_qty
                cash += filled_qty * price
        cooldown_timer[symbol] = current_time

    if recommendations:
        print("Recommended Actions:")
        buys = sorted([r for r in recommendations if r[2] == "BUY"], reverse=True)[:2]
        sells = sorted([r for r in recommendations if r[2] == "SELL"], reverse=True)[:2]
        for group in [buys, sells]:
            for score, symbol, side, price, qty, strategy_type in group:
                flag = "urgent" if score >= 5 else ("important" if score >= 3 else "wait")
                print(f"{flag} {symbol} {side} {qty} @ {round(price, 2)} | Score: {score} | Mode: {strategy_type}")

def on_error(ws, error):
    print("WebSocket Error:", repr(error))

def on_close(ws, close_status_code, close_msg):
    print(f"WebSocket closed with code={close_status_code}, message={close_msg}")

def on_open(ws):
    print("WebSocket connection opened")

url = "ws://localhost:8765"
ws = websocket.WebSocketApp(url, on_message=on_message, on_error=on_error, on_close=on_close)
ws.on_open = on_open
ws.run_forever()
